In [0]:
# Code tested in google colab with T4 GPU

In [0]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2

In [0]:
import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import json

In [0]:
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [0]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [0]:
# Utility Functions ::

def get_tokenizer(model_name):
  """
  Args:
    model_name: LLM model name

  Returns:
    Tokenizer
  """
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  tokenizer.pad_token = tokenizer.eos_token

  return tokenizer


def get_model(model_name):
    """
    Args:
      model_name: LLM model name

    Returns:
      Model
    """
    quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_quant_type="nf4"
            )

    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=quant_config)

    return model



In [0]:
tokenizer = get_tokenizer(LLAMA)
model = get_model(LLAMA)

In [0]:

def parse_response(inputs, outputs):
  """
  Args:
    inputs: Input Tokens, pt tensors
    outputs: Model generated output token

  Returns:
    response
  """
  full_input =(tokenizer.batch_decode(inputs, skip_special_tokens=True)[0])
  full_outputs=tokenizer.batch_decode(outputs, skip_special_tokens=True)
  response = full_outputs[0][len(full_input):]

  return response

# Tools Functions:

def get_tools_response(tools_params):
  function_name = tools_params['name']
  arguments = tools_params['arguments']
  location = arguments['location']

  if function_name  == 'get_current_temperature':
    response = get_current_temperature(location)
    return response
  elif function_name  == 'get_current_wind_speed':
    response = get_current_wind_speed(location)
    return response


def get_current_temperature(location: str) -> float:
    """
    Get the current temperature at a location.

    Args:
        location: The location to get the temperature for, in the format "City, Country"
    Returns:
        The current temperature at the specified location in the specified units, as a float.
    """
    if location ==  "Bangalore, India":  
      return 22 # for testing purpose, please replace with your logic


def get_current_wind_speed(location: str) -> str:
    """
    Get the current wind speed at a given location.

    Args:
        location: The location to get the wind speed for, in the format "City, Country"
    Returns:
        The current wind speed at the given location in the specified units, as a string.
    """
    return 6 # for testing purpose, please replace with your logic


def get_llm_response(message):

    messages = [
              {"role": "system", "content": """
                        You are an helpful assistant that responds to weather and wind queries. Please provide accurate answers.
                        Follow instructions:
                        1. Please provide courteous answers.
                        2. If you do not know the answer say so.
                        3. Do not provide any explanations or suggestions.
                        4. Provide the response in no more than 1 sentence.
                    """
                    },
    ]
    message = {"role": "user", "content": message}

    messages.append(message)


    inputs = tokenizer.apply_chat_template(messages, tools=[get_current_temperature, get_current_wind_speed], add_generation_prompt=True, return_tensors="pt").to("cuda")

    #streamer = TextStreamer(tokenizer) #  include to check the output of model, only for debug
    outputs = model.generate(inputs, max_new_tokens=2000 )#, streamer=streamer ) # , include to check the output of model, only for debug

    model_response = parse_response(inputs, outputs)

    tool_call = json.loads(model_response.replace("parameters", "arguments"))
    messages.append({"role": "assistant", "tool_calls": [{"type": "function", "function": tool_call}]})

    response = get_tools_response(tool_call)

    messages.append({"role": "tool", "name": tool_call["name"], "content": response})

    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to("cuda")
    outputs = model.generate(inputs, max_new_tokens=200)

    response = parse_response(inputs, outputs)

    return response




In [0]:


data = [
    "what is the wind speed in Bangalore?",
    "what is the temperature of Bangalore?",
    "temperature in Delhi?",
    "wind speed in Goa?",
    "Chennai's temperature?",
]

for query in data:
  response = get_llm_response(query)
  print("#"*100)
  print(f"Query: {query}\nResponse: {response}\n\n")

